<a href="https://colab.research.google.com/github/mikeogunmakin/monzo_1p_challenge_tracker/blob/main/ld_rfm_analysis_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RFM Analysis in Python

**Customer segmentation** is the practice of grouping customers based on common characteristics. These customer segments are beneficial in marketing campaigns, in identifying potentially profitable customers, and in developing customer loyalty. A company might segment customers according to a wide range of factors, including: demographics (age, gender, location etc), behaviour (previous orders, responses to messaging), psychographics (values, interests, lifestyles) etc.
<br><br>

**RFM (Recency-Frequency-Monetary) analysis** is a simple technique for behaviour based customer segmentation. It groups customers based on their transaction history – how recently, how often and how much did they buy. It is a handy method to find the best customers, understand their behavior and then run targeted marketing campaigns to increase sales, satisfaction and customer lifetime value.

The goal of this analysis is to identify customer segments for global retail company using RFM analysis and to understand how those groups differ from each other.

In [1]:
!pip install squarify

# import modules
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import squarify
import json

import geopandas as gpd
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer
from bokeh.models import HoverTool

pd.options.display.float_format = '{:,.2f}'.format ## add to notes

In [2]:
# reading in the file
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/202501 RFM Analysis/superstore_dataset2011-2015.csv', encoding="latin-1") ## add to notes

df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,42433,AG-2011-2040,1/1/2011,6/1/2011,Standard Class,TB-11280,Toby Braunhardt,Consumer,Constantine,Constantine,...,OFF-TEN-10000025,Office Supplies,Storage,"Tenex Lockers, Blue",408.30,2,0.00,106.14,35.46,Medium
1,22253,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,...,OFF-SU-10000618,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.37,3,0.10,36.04,9.72,Medium
2,48883,HU-2011-1220,1/1/2011,5/1/2011,Second Class,AT-735,Annie Thurman,Consumer,Budapest,Budapest,...,OFF-TEN-10001585,Office Supplies,Storage,"Tenex Box, Single Width",66.12,4,0.00,29.64,8.17,High
3,11731,IT-2011-3647632,1/1/2011,5/1/2011,Second Class,EM-14140,Eugene Moren,Home Office,Stockholm,Stockholm,...,OFF-PA-10001492,Office Supplies,Paper,"Enermax Note Cards, Premium",44.87,3,0.50,-26.05,4.82,High
4,22255,IN-2011-47883,1/1/2011,8/1/2011,Standard Class,JH-15985,Joseph Holt,Consumer,Wagga Wagga,New South Wales,...,FUR-FU-10003447,Furniture,Furnishings,"Eldon Light Bulb, Duo Pack",113.67,5,0.10,37.77,4.70,Medium


In [3]:
#inspect the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Row ID          51290 non-null  int64  
 1   Order ID        51290 non-null  object 
 2   Order Date      51290 non-null  object 
 3   Ship Date       51290 non-null  object 
 4   Ship Mode       51290 non-null  object 
 5   Customer ID     51290 non-null  object 
 6   Customer Name   51290 non-null  object 
 7   Segment         51290 non-null  object 
 8   City            51290 non-null  object 
 9   State           51290 non-null  object 
 10  Country         51290 non-null  object 
 11  Postal Code     9994 non-null   float64
 12  Market          51290 non-null  object 
 13  Region          51290 non-null  object 
 14  Product ID      51290 non-null  object 
 15  Category        51290 non-null  object 
 16  Sub-Category    51290 non-null  object 
 17  Product Name    51290 non-null 

In [4]:
df.columns = [col.lower().replace(' ', '_') for col in df.columns] ## add to notes

df = df.drop(['row_id', 'ship_date', 'customer_name', 'postal_code', 'shipping_cost', 'order_priority'], axis=1) ## add to notes


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   order_id      51290 non-null  object 
 1   order_date    51290 non-null  object 
 2   ship_mode     51290 non-null  object 
 3   customer_id   51290 non-null  object 
 4   segment       51290 non-null  object 
 5   city          51290 non-null  object 
 6   state         51290 non-null  object 
 7   country       51290 non-null  object 
 8   market        51290 non-null  object 
 9   region        51290 non-null  object 
 10  product_id    51290 non-null  object 
 11  category      51290 non-null  object 
 12  sub-category  51290 non-null  object 
 13  product_name  51290 non-null  object 
 14  sales         51290 non-null  float64
 15  quantity      51290 non-null  int64  
 16  discount      51290 non-null  float64
 17  profit        51290 non-null  float64
dtypes: float64(3), int64(1), o

In [20]:
df['order_date'] = df['order_date'].str.replace('-','/')

In [24]:
df['order_date'] = pd.to_datetime(df['order_date'], format = "%d/%m/%Y") ## add to notes

df['order_date'].head()

In [26]:
# Analysis as of: 2015-01-31 (max order date in the dataset: 2014-12-31)
today = datetime.strptime('2015-01-31', '%Y-%m-%d') #set as reference date

In [38]:


agg_dict1 = {
    'order_id': 'count',
    'order_date': 'max',
    'sales': 'sum'
}

df_rfm = df.groupby('customer_id').agg(agg_dict1).reset_index()

df_rfm.columns = ['customer_id', 'frequency', 'max_date', 'monetary']

df_rfm['recency'] = (today - df_rfm['max_date']).dt.days

df_rfm
# df_rfm.drop(['max_date'], axis=1, inplace=True)

,customer_id,frequency,max_date,monetary,recency
0,AA-10315,42,2014-12-23,"13,747.41",39
1,AA-10375,42,2014-12-25,"5,884.20",37
2,AA-10480,38,2014-08-28,"17,695.59",156
3,AA-10645,73,2014-12-03,"15,343.89",59
4,AA-315,8,2014-12-29,"2,243.26",33
...,...,...,...,...,...
1585,YS-21880,54,2014-12-22,"18,703.61",40
1586,ZC-11910,1,2014-06-14,7.17,231
1587,ZC-21910,84,2014-12-28,"28,472.82",34
1588,ZD-11925,18,2014-12-28,"2,951.23",34


In [32]:
df_copy = df.copy()

df_copy.head(3)

,order_id,order_date,ship_mode,customer_id,segment,city,state,country,market,region,product_id,category,sub-category,product_name,sales,quantity,discount,profit
0,AG-2011-2040,2011-01-01,Standard Class,TB-11280,Consumer,Constantine,Constantine,Algeria,Africa,Africa,OFF-TEN-10000025,Office Supplies,Storage,"Tenex Lockers, Blue",408.30,2,0.00,106.14
1,IN-2011-47883,2011-01-01,Standard Class,JH-15985,Consumer,Wagga Wagga,New South Wales,Australia,APAC,Oceania,OFF-SU-10000618,Office Supplies,Supplies,"Acme Trimmer, High Speed",120.37,3,0.10,36.04
2,HU-2011-1220,2011-01-01,Second Class,AT-735,Consumer,Budapest,Budapest,Hungary,EMEA,EMEA,OFF-TEN-10001585,Office Supplies,Storage,"Tenex Box, Single Width",66.12,4,0.00,29.64


In [40]:
# from datetime import datetime,timedelta

df_rfm2 = df_copy.groupby('customer_id').agg({
     'order_date': lambda x: (today - x.max()).days,
     'order_id' : 'count',
     'sales' : 'sum'

})


df_rfm2.head(10)

,order_date,order_id,sales
customer_id,,,
AA-10315,39,42,"13,747.41"
AA-10375,37,42,"5,884.20"
AA-10480,156,38,"17,695.59"
AA-10645,59,73,"15,343.89"
AA-315,33,8,"2,243.26"
AA-375,284,13,654.49
AA-480,345,10,"2,063.49"
AA-645,82,18,"1,968.32"
AB-10015,47,77,"20,037.50"
